In [3]:
import os
data_dir = os.path.expanduser("~/Google Drive/Shared drives/Data")
data_path = f'{data_dir}/BEIR/bioasq'

In [4]:
from vectorspace.utils import jsonl_load, json_load
corpus = json_load(f"{data_path}/reduced_corpus.json")
sentences = ['\n'.join([doc["title"], doc["text"]]).strip() if doc.get("title") else doc["text"].strip()
            for doc in corpus.values()]

In [5]:
from beir.datasets.data_loader import GenericDataLoader
results = json_load(f"{data_dir}/BEIR/results_bioasq_bm25.json")
_, queries, qrels = GenericDataLoader(data_path).load(split="test")

/opt/homebrew/anaconda3/lib/python3.9/site-packages/beir/datasets/data_loader.py:2: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


  0%|          | 0/1 [00:00<?, ?it/s]

In [6]:
from vectorspace.embedding import Embedding

In [31]:
paths = [f"{data_path}/instructor_large_reduced_corpus_{start}-{start+10000}" for start in range(0, 380000, 10000)]
embs = [Embedding.load(p) for p in paths]
vecs = np.concatenate([emb.vecs for emb in embs])

In [55]:
words = [w for emb in embs for w in list(emb.words)]

In [67]:
corpus_emb = Embedding.create(vecs, words)

In [73]:
corpus_emb.dump(f"{data_path}/instructor_large_reduced_corpus")

In [68]:
#sims = corpus_emb.knn('29434076')
#print(corpus['29434076'])
#print()
#for d in sims[0]:
#    print(corpus[d])

In [69]:
#!git clone https://github.com/HKUNLP/instructor-embedding
#!pip install instructor-embedding/sentence-transformers

In [64]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('hkunlp/instructor-large')

max_seq_length  512


In [65]:
prompt = "Represent the Science question for retrieving supporting paragraphs; Input: "

In [71]:
from sentence_transformers import util

In [ ]:
results_rerank = {query_id: {} for query_id in results}
for query_id, doc_ids in results.items():
    q = queries[query_id] 
    query_vec = model.encode([prompt, q, 0])[0]
    doc_ids = list(results[query_id].keys())
    doc_vecs = corpus_emb[doc_ids[:1000]]
    query_vec = model.encode([queries[query_id]])[0]
    scores = util.cos_sim(query_vec, doc_vecs)[0]
    for doc_id, score in zip(doc_ids, scores):
        results_rerank[query_id][doc_id] = float(score)

In [ ]:
results_rerank['5e29fb27aa19d74431000005']

In [ ]:
from beir.retrieval.evaluation import EvaluateRetrieval

In [ ]:
ndcg, _map, recall, precision = EvaluateRetrieval.evaluate(qrels, results_rerank, k_values=[1, 3, 5, 10, 100, 1000])